# Задание 1

**Для каждой организации из main.csv подтяните выручку и число сотрудников за 2020, 2021 и 2022:**
* посчитайте дельту по выручке и сотрудникам между 2022 и 2020;
* отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID;
* если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их;
* названия колонок сделайте на русском языке.

## Описание датасетов

#### main.csv - *основная информация о предприятиях*
- ID - уникальный номер объекта;
- Name - название;
- Address - адрес

#### net_property.csv - *указание на сетевые предприятия*
- ID - уникальный номер объекта;
- IsNetObject - метка сетевого предприятия;

#### seats_property.csv - *количество посадочных мест*
- ID - уникальный номер объекта;
- SeatsCount - количество посадочных мест;
- ver_ID - номер версии записи

#### type_property.csv - *информация о специализации: столовая, закусочная, кафе, иное*
- ID - уникальный номер объекта;
- TypeObject - тип объекта;
- is_actual - актуальность записи

#### indicators_db.sqlite3 - *SQL-база с данными о выручке предприятий и числе сотрудников*
- данные за 2019 г.;
- данные за 2020-2022 гг.

In [9]:
import pandas as pd
import numpy as np

### Чтение датасетов

In [10]:
df1 = pd.read_csv('main.csv')
df2 = pd.read_csv('net_property.csv')
df3 = pd.read_csv('seats_property.csv')
df4 = pd.read_csv('type_property.csv')

In [3]:
import sqlalchemy as sql

In [4]:
con = sql.create_engine('sqlite:///indicators_db.sqlite3')

In [5]:
df = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)

In [6]:
df

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...


In [7]:
df_sql_select_20_22 = pd.read_sql('select * from org_indicators_2020_2022', con)

In [156]:
df_sql_select_20_22

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


### Выборка необходимых колонок из SQL-базы за 2020-2022 и разворот через pivot

In [9]:
df_20_22 = df_sql_select_20_22[(df_sql_select_20_22['INDICATOR_NAME'] == 'Число сотрудников') | \
                    (df_sql_select_20_22['INDICATOR_NAME']  == 'Выручка')]

In [161]:
df_20_22_piv = df_20_22.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [162]:
df_20_22_piv

YEAR                  2020                          2021                    \
INDICATOR_NAME     Выручка Число сотрудников     Выручка Число сотрудников   
ID                                                                           
19630           31222228.0               NaN  21721921.0              15.0   
19643           35670701.0              17.0  39125612.0               NaN   
19647           34926614.0              16.0  23176004.0              15.0   
19657           23410468.0               NaN  28065717.0               8.0   
19677                  NaN              17.0  30803884.0               9.0   
...                    ...               ...         ...               ...   
375884                 NaN               8.0  32408516.0              10.0   
375933          36102422.0               NaN  21245735.0              16.0   
375954                 NaN              16.0         NaN               NaN   
375956                 NaN               NaN  32178479.0               NaN   
375960                 NaN              13.0  39567594.0               NaN   

YEAR                        2022              
INDICATOR_NAME Число сотрудников     Выручка  
ID                                            
19630                        9.0         NaN  
19643                        NaN         NaN  
19647                        NaN  27925131.0  
19657                       12.0  36074059.0  
19677                        NaN  25303592.0  
...                          ...         ...  
375884                       7.0         NaN  
375933                       NaN  39298414.0  
375954                      11.0  20002278.0  
375956                       6.0         NaN  
375960                       NaN  37986011.0  

[10000 rows x 6 columns]

### Удаление уровня "YEAR" и переименование колонок

In [231]:
testing = df_20_22_piv.droplevel([0], axis = 1)

In [232]:
testing.columns.names = [None]

In [233]:
testing.to_csv('test.csv')

In [234]:
testing = pd.read_csv('test.csv')

In [235]:
rename = {'Выручка':'Выручка 2020', 'Число сотрудников':'Число сотрудников 2020',
          'Выручка.1':'Выручка 2021', 'Число сотрудников.1':'Число сотрудников 2021',
          'Выручка.2':'Выручка 2022', 'Число сотрудников.2':'Число сотрудников 2022'}

In [236]:
testing = testing.rename(columns = rename)

In [237]:
testing

,ID,Выручка 2020,Число сотрудников 2020,Выручка 2021,Число сотрудников 2021,Число сотрудников 2022,Выручка 2022
0,19630,31222228.0,NaN,21721921.0,15.0,9.0,NaN
1,19643,35670701.0,17.0,39125612.0,NaN,NaN,NaN
2,19647,34926614.0,16.0,23176004.0,15.0,NaN,27925131.0
3,19657,23410468.0,NaN,28065717.0,8.0,12.0,36074059.0
4,19677,NaN,17.0,30803884.0,9.0,NaN,25303592.0
...,...,...,...,...,...,...,...
9995,375884,NaN,8.0,32408516.0,10.0,7.0,NaN
9996,375933,36102422.0,NaN,21245735.0,16.0,NaN,39298414.0
9997,375954,NaN,16.0,NaN,NaN,11.0,20002278.0
9998,375956,NaN,NaN,32178479.0,NaN,6.0,NaN


### Вычисление дельты и изменение порядка колонок

In [238]:
testing['Дельта по выручке'] = testing['Выручка 2022'] - testing['Выручка 2020']
testing['Дельта по сотрудникам'] = testing['Число сотрудников 2022'] - testing['Число сотрудников 2020']

In [239]:
testing

,ID,Выручка 2020,Число сотрудников 2020,Выручка 2021,Число сотрудников 2021,Число сотрудников 2022,Выручка 2022,Дельта по выручке,Дельта по сотрудникам
0,19630,31222228.0,NaN,21721921.0,15.0,9.0,NaN,NaN,NaN
1,19643,35670701.0,17.0,39125612.0,NaN,NaN,NaN,NaN,NaN
2,19647,34926614.0,16.0,23176004.0,15.0,NaN,27925131.0,-7001483.0,NaN
3,19657,23410468.0,NaN,28065717.0,8.0,12.0,36074059.0,12663591.0,NaN
4,19677,NaN,17.0,30803884.0,9.0,NaN,25303592.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9995,375884,NaN,8.0,32408516.0,10.0,7.0,NaN,NaN,-1.0
9996,375933,36102422.0,NaN,21245735.0,16.0,NaN,39298414.0,3195992.0,NaN
9997,375954,NaN,16.0,NaN,NaN,11.0,20002278.0,NaN,-5.0
9998,375956,NaN,NaN,32178479.0,NaN,6.0,NaN,NaN,NaN


### Удаление пропусков и лишних колонок в годах 2020 и 2022, полученных в ходе объединения

In [247]:
mask = testing[['ID','Выручка 2020','Выручка 2022', 'Число сотрудников 2020','Число сотрудников 2022']].dropna()

In [263]:
merge_testing = pd.merge(mask, testing, how = 'inner', on = 'ID')

In [264]:
merge_testing

,ID,Выручка 2020_x,Выручка 2022_x,Число сотрудников 2020_x,Число сотрудников 2022_x,Выручка 2020_y,Выручка 2021,Выручка 2022_y,Дельта по выручке,Число сотрудников 2020_y,Число сотрудников 2021,Число сотрудников 2022_y,Дельта по сотрудникам
0,19726,34342910.0,37135202.0,11.0,8.0,34342910.0,39073727.0,37135202.0,2792292.0,11.0,8.0,8.0,-3.0
1,19727,26257455.0,38039622.0,8.0,11.0,26257455.0,27300658.0,38039622.0,11782167.0,8.0,NaN,11.0,3.0
2,19741,26616852.0,27339029.0,11.0,14.0,26616852.0,33580915.0,27339029.0,722177.0,11.0,12.0,14.0,3.0
3,19823,27222724.0,22123140.0,12.0,10.0,27222724.0,30561355.0,22123140.0,-5099584.0,12.0,NaN,10.0,-2.0
4,19939,34539627.0,25711365.0,12.0,18.0,34539627.0,NaN,25711365.0,-8828262.0,12.0,12.0,18.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581,375289,27078398.0,28646290.0,13.0,7.0,27078398.0,34867524.0,28646290.0,1567892.0,13.0,NaN,7.0,-6.0
1582,375352,31787477.0,37116392.0,11.0,13.0,31787477.0,NaN,37116392.0,5328915.0,11.0,15.0,13.0,2.0
1583,375629,26926434.0,29411964.0,14.0,11.0,26926434.0,24479012.0,29411964.0,2485530.0,14.0,12.0,11.0,-3.0
1584,375654,35269880.0,35309041.0,18.0,13.0,35269880.0,24593454.0,35309041.0,39161.0,18.0,13.0,13.0,-5.0


In [266]:
merge_testing.drop(['Выручка 2020_y', 'Выручка 2022_y',
                    'Число сотрудников 2020_y','Число сотрудников 2022_y'], axis = 1, inplace = True)

In [268]:
merge_testing

,ID,Выручка 2020_x,Выручка 2022_x,Число сотрудников 2020_x,Число сотрудников 2022_x,Выручка 2021,Дельта по выручке,Число сотрудников 2021,Дельта по сотрудникам
0,19726,34342910.0,37135202.0,11.0,8.0,39073727.0,2792292.0,8.0,-3.0
1,19727,26257455.0,38039622.0,8.0,11.0,27300658.0,11782167.0,NaN,3.0
2,19741,26616852.0,27339029.0,11.0,14.0,33580915.0,722177.0,12.0,3.0
3,19823,27222724.0,22123140.0,12.0,10.0,30561355.0,-5099584.0,NaN,-2.0
4,19939,34539627.0,25711365.0,12.0,18.0,NaN,-8828262.0,12.0,6.0
...,...,...,...,...,...,...,...,...,...
1581,375289,27078398.0,28646290.0,13.0,7.0,34867524.0,1567892.0,NaN,-6.0
1582,375352,31787477.0,37116392.0,11.0,13.0,NaN,5328915.0,15.0,2.0
1583,375629,26926434.0,29411964.0,14.0,11.0,24479012.0,2485530.0,12.0,-3.0
1584,375654,35269880.0,35309041.0,18.0,13.0,24593454.0,39161.0,13.0,-5.0


In [270]:
rename_2 = {'Выручка 2020_x':'Выручка 2020',
            'Выручка 2022_x':'Выручка 2022',
            'Число сотрудников 2020_x':'Число сотрудников 2020',
            'Число сотрудников 2022_x':'Число сотрудников 2022'}
merge_testing = merge_testing.rename(columns = rename_2)

In [311]:
merge_testing = merge_testing[['ID','Выручка 2020','Выручка 2021','Выручка 2022', 'Дельта по выручке',
         'Число сотрудников 2020','Число сотрудников 2021','Число сотрудников 2022', 'Дельта по сотрудникам']]

In [312]:
merge_testing

,ID,Выручка 2020,Выручка 2021,Выручка 2022,Дельта по выручке,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022,Дельта по сотрудникам
0,19726,34342910.0,39073727.0,37135202.0,2792292.0,11.0,8.0,8.0,-3.0
1,19727,26257455.0,27300658.0,38039622.0,11782167.0,8.0,NaN,11.0,3.0
2,19741,26616852.0,33580915.0,27339029.0,722177.0,11.0,12.0,14.0,3.0
3,19823,27222724.0,30561355.0,22123140.0,-5099584.0,12.0,NaN,10.0,-2.0
4,19939,34539627.0,NaN,25711365.0,-8828262.0,12.0,12.0,18.0,6.0
...,...,...,...,...,...,...,...,...,...
1581,375289,27078398.0,34867524.0,28646290.0,1567892.0,13.0,NaN,7.0,-6.0
1582,375352,31787477.0,NaN,37116392.0,5328915.0,11.0,15.0,13.0,2.0
1583,375629,26926434.0,24479012.0,29411964.0,2485530.0,14.0,12.0,11.0,-3.0
1584,375654,35269880.0,24593454.0,35309041.0,39161.0,18.0,13.0,13.0,-5.0


In [313]:
sorted_testing = merge_testing.sort_values(by = 'Дельта по выручке', ascending = False)

### Добавление основного датасета, переименование и сохранение

In [314]:
df_testing = pd.merge(df1,sorted_testing, on = 'ID', how = 'inner')

In [315]:
rename_3 = {'ID':'Идентификатор',
            'Name':'Наименование',
            'Address':'Адрес'}
df_testing = df_testing.rename(columns = rename_3)

In [319]:
df_test_sort = df_testing.sort_values(by = 'Дельта по выручке', ascending = False)

In [321]:
df_test_sort.to_csv('final.csv', index=False)

### Проверка результирующего датасета

In [322]:
pd.read_csv('final.csv')

,Идентификатор,Наименование,Адрес,Выручка 2020,Выручка 2021,Выручка 2022,Дельта по выручке,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022,Дельта по сотрудникам
0,342920,СИДРОВ ПРОЛИВ (пивной бар),"город Москва, проспект Вернадского, дом 86В",20302399.0,NaN,39959846.0,19657447.0,6.0,11.0,14.0,8.0
1,374427,Море лосося,"город Москва, улица Адмирала Макарова, дом 6, ...",20000135.0,NaN,39484937.0,19484802.0,7.0,NaN,19.0,12.0
2,360584,"Выпечка, шаурма","Российская Федерация, город Москва, внутригоро...",20114682.0,23403344.0,39418028.0,19303346.0,13.0,NaN,11.0,-2.0
3,344731,Кофе с собой,"Российская Федерация, город Москва, внутригоро...",21020009.0,39682516.0,39851594.0,18831585.0,8.0,NaN,10.0,2.0
4,373914,Кафе,"город Москва, улица Айвазовского, владение 7, ...",20663231.0,31079091.0,38834892.0,18171661.0,7.0,16.0,14.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
1581,129686,Три правила,"город Москва, улица Мичуринский Проспект, Олим...",39778987.0,NaN,21141214.0,-18637773.0,15.0,17.0,8.0,-7.0
1582,344859,Хлеб с маслом,"город Москва, Манежная площадь, дом 1, строение 2",38772086.0,NaN,20065466.0,-18706620.0,13.0,8.0,15.0,2.0
1583,150865,Lost Bar,"город Москва, Новокосинская улица, дом 22",39338589.0,34587746.0,20463995.0,-18874594.0,15.0,18.0,15.0,0.0
1584,239777,Surf Coffee,"город Москва, Берсеневская набережная, дом 14,...",39515097.0,28540582.0,20432343.0,-19082754.0,19.0,NaN,9.0,-10.0
